# Correlation

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats

FOLDER = "merged3/"
FULL_NAME = "_max_30_days.csv"

targets = ['likes', 'comments']
features = ['is_sponsored', 'is_video', 'days_within', 'followers', 'tier', 'nudity_0', 'nudity_1', 'nudity_2', 'nudity_3', 'nudity_4', 'nudity_5', 'nudity_6', 'nudity_7', 'nudity_8', 'nudity_9', 'nudity_10', 'nudity_11', 'nudity_12', 'nudity_13', 'nudity_14', 'nudity_15', 'r_percent', 'g_percent', 'b_percent', 'lum_percent', 'percent_warm', 'percent_cold', 'pleasure', 'arousal', 'dominance', 'places365_macro', 'places365_micro', 'person', 'vehicle', 'outdoor', 'animal', 'accessory', 'sports', 'kitchen', 'food', 'furniture', 'electronic', 'appliance', 'indoor', 'n_emojis', 'weighted_emoji_sent', 'n_hashtags', 'n_mentions', 'is_location', 'caption_len',"n_male", "n_female", "mean_age", "std_age", "min_age", "max_age", 
                               "mean_male_age", "std_male_age", "mean_female_age", "std_female_age",]

In [ ]:
NUDENET_CLASSES = {0: 'EXPOSED_ANUS',
                     1: 'EXPOSED_ARMPITS',
                     2: 'COVERED_BELLY',
                     3: 'EXPOSED_BELLY',
                     4: 'COVERED_BUTTOCKS',
                     5: 'EXPOSED_BUTTOCKS',
                     6: 'FACE_F',
                     7: 'FACE_M', 
                     8: 'COVERED_FEET',
                     9: 'EXPOSED_FEET',
                     10: 'COVERED_BREAST_F',
                     11: 'EXPOSED_BREAST_F',
                     12: 'COVERED_GENITALIA_F',
                     13: 'EXPOSED_GENITALIA_F',
                     14: 'EXPOSED_BREAST_M',
                     15: 'EXPOSED_GENITALIA_M'}


places365_macro = {0: "indoor",
                   1: "outdoor_natural",
                   2: "outdoor_manmade"
            }

places365_micro = {0:"shopping_dining", 1:"workplace",2:"home_hotel", 3:"transportation", 4:"sports_leisure", 5:"cultural",
                  6:"water_ice_snow", 7:"mountains_hills_desert_sky", 8:"forest_field_jungle", 9:"man-made_elements",
                  10:"transportations", 11:"cultural_historical", 12:"sports_leisure_parks", 13:"industrial_costruction", 14:"houses_cabins_gardens_farms", 15:"commercial_buildings"}

## Likes

In [ ]:
target = "like/fol"


correlation_dfs = []

for category in ["beauty", "fashion", "fitness", "family", "pet", "food", "interior", "other", "travel"]:
    
    features = ['is_sponsored', 'is_video',
            'days_within',
            'nudity_0', 'nudity_1', 'nudity_2', 'nudity_3', 'nudity_4', 'nudity_5', 'nudity_6', 
            'nudity_7', 'nudity_8', 'nudity_9', 'nudity_10', 'nudity_11', 'nudity_12', 'nudity_13', 'nudity_14', 'nudity_15', 'r_percent', 'g_percent', 
            'b_percent', 'lum_percent', 'percent_warm', 'percent_cold', 'pleasure', 'arousal', 'dominance',
            'person', 'vehicle', 'outdoor', 'animal', 'accessory', 'sports', 'kitchen', 'food', 'furniture', 'electronic', 'appliance', 'indoor', 'n_emojis', 
            'weighted_emoji_sent', 'n_hashtags', 'n_mentions', 'is_location', 
            'caption_len',"n_male", "n_female", "mean_age", "std_age", "min_age", "max_age", 
            "mean_male_age", "std_male_age", "mean_female_age", "std_female_age", "cap_sentiment_score", "cap_sentiment_magnitude",
            "ImgSentimentScore", "ColorHarmony", "MotionBlur", "Light", "Symmetry", "Repetition", "Aest_score", 
            "DoF", "VividColor", "Content", "Object", "BalancingElement", "RuleOfThirds"
               ]
    
    df = pd.read_csv(f'{FOLDER}{category}{FULL_NAME}')
    df = df.loc[df.cap_sentiment_magnitude != -1]    
    
    df['weighted_emoji_sent'] = df['weighted_emoji_sent'].fillna(0)
    
    df["places365_macro"] = df["places365_macro"].astype(str)
    df["places365_micro"] = df["places365_micro"].astype(str)
    
    # dummy variables for places macro/micro
    dummies = pd.get_dummies(df['places365_macro']).rename(columns=lambda x: 'places365_macro_' + str(x))
    features.extend(dummies.columns.tolist())
    df = pd.concat([df, dummies], axis=1)
    df.drop(['places365_macro'], inplace=True, axis=1)
        
    dummies = pd.get_dummies(df['places365_micro']).rename(columns=lambda x: 'places365_micro_' + str(x))
    features.extend(dummies.columns.tolist())
    df = pd.concat([df, dummies], axis=1)
    df.drop(['places365_micro'], inplace=True, axis=1)


    print(category)
    
    for tier, tier_df in df.groupby(by="tier"):
        pearson_corr = pd.DataFrame(columns=["feature", "corr", "p-value"])

        curr_tier = list(tiers.keys())[list(tiers.values()).index(tier)]

        print("\t", curr_tier)
        for feature in features + ["cuteness"]:
            if feature == "cuteness" and category != "pet":
                continue
            
            if "nudity" in feature and category not in ["fashion", "beauty"]:
                continue 

            target_values = tier_df[target].values
            feature_values = tier_df[feature]

            corr, p_value = scipy.stats.spearmanr(feature_values, target_values)
            
            if "places365_macro" in feature:
                macro = places365_macro[int(float(feature.split("_")[-1]))]
                feature = f'places_macro_{macro}'
            elif "places365_micro" in feature:
                micro = places365_micro[int(float(feature.split("_")[-1]))]
                feature = f'places_micro_{micro}'
            elif "nudity" in feature:
                n = NUDENET_CLASSES[int(float(feature.split("_")[-1]))]
                feature = f'nudity_{n}'
                                  
            if str(corr) != "nan":
                pearson_corr.loc[len(pearson_corr)] = {"feature":feature, "corr":corr, "p-value":p_value}

        pearson_corr.sort_values(by="corr", ascending=False, inplace=True)
        pearson_corr.to_csv(f'correlations/{target.replace("/", "_")}/{category}_tier_{curr_tier}.csv', index=False)
        correlation_dfs.append(pearson_corr)

## Comments

In [ ]:
target = "comm/fol"

correlation_dfs = []

for category in ["beauty", "fashion", "fitness", "family", "pet", "food", "interior", "other", "travel"]:
    
    features = ['is_sponsored', 'is_video',
            'days_within',
            'nudity_0', 'nudity_1', 'nudity_2', 'nudity_3', 'nudity_4', 'nudity_5', 'nudity_6', 
            'nudity_7', 'nudity_8', 'nudity_9', 'nudity_10', 'nudity_11', 'nudity_12', 'nudity_13', 'nudity_14', 'nudity_15', 'r_percent', 'g_percent', 
            'b_percent', 'lum_percent', 'percent_warm', 'percent_cold', 'pleasure', 'arousal', 'dominance',
            'person', 'vehicle', 'outdoor', 'animal', 'accessory', 'sports', 'kitchen', 'food', 'furniture', 'electronic', 'appliance', 'indoor', 'n_emojis', 
            'weighted_emoji_sent', 'n_hashtags', 'n_mentions', 'is_location', 
            'caption_len',"n_male", "n_female", "mean_age", "std_age", "min_age", "max_age", 
            "mean_male_age", "std_male_age", "mean_female_age", "std_female_age", "cap_sentiment_score", "cap_sentiment_magnitude",
            "ImgSentimentScore", "ColorHarmony", "MotionBlur", "Light", "Symmetry", "Repetition", "Aest_score", 
                               "DoF", "VividColor", "Content", "Object", "BalancingElement", "RuleOfThirds",
               ]
    
    df = pd.read_csv(f'{FOLDER}{category}{FULL_NAME}')
    df = df.loc[df.cap_sentiment_magnitude != -1]

    df['weighted_emoji_sent'] = df['weighted_emoji_sent'].fillna(0)
    
    df["places365_macro"] = df["places365_macro"].astype(str)
    df["places365_micro"] = df["places365_micro"].astype(str)
    
    # dummy variables for places macro/micro
    dummies = pd.get_dummies(df['places365_macro']).rename(columns=lambda x: 'places365_macro_' + str(x))
    features.extend(dummies.columns.tolist())
    df = pd.concat([df, dummies], axis=1)
    df.drop(['places365_macro'], inplace=True, axis=1)
        
    dummies = pd.get_dummies(df['places365_micro']).rename(columns=lambda x: 'places365_micro_' + str(x))
    features.extend(dummies.columns.tolist())
    df = pd.concat([df, dummies], axis=1)
    df.drop(['places365_micro'], inplace=True, axis=1)


    print(category)
    
    for tier, tier_df in df.groupby(by="tier"):
        pearson_corr = pd.DataFrame(columns=["feature", "corr", "p-value"])

        curr_tier = list(tiers.keys())[list(tiers.values()).index(tier)]

        print("\t", curr_tier)
        for feature in features + ["cuteness"]:
            if feature == "cuteness" and category != "pet":
                continue
            
            if "nudity" in feature and category not in ["fashion", "beauty"]:
                continue 

            target_values = tier_df[target].values
            feature_values = tier_df[feature]

            corr, p_value = scipy.stats.spearmanr(feature_values, target_values)
            
            if "places365_macro" in feature:
                macro = places365_macro[int(float(feature.split("_")[-1]))]
                feature = f'places_macro_{macro}'
            elif "places365_micro" in feature:
                micro = places365_micro[int(float(feature.split("_")[-1]))]
                feature = f'places_micro_{micro}'
            elif "nudity" in feature:
                n = NUDENET_CLASSES[int(float(feature.split("_")[-1]))]
                feature = f'nudity_{n}'
                                  
            if str(corr) != "nan":
                pearson_corr.loc[len(pearson_corr)] = {"feature":feature, "corr":corr, "p-value":p_value}

        pearson_corr.sort_values(by="corr", ascending=False, inplace=True)
        pearson_corr.to_csv(f'correlations/{target.replace("/", "_")}/{category}_tier_{curr_tier}.csv', index=False)
        correlation_dfs.append(pearson_corr)